In [1]:
import pandas as pd

In [37]:
df_jan = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
# df = pd.concat([df_jan, df_feb], axis=0, ignore_index=True)

In [38]:
df_jan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [39]:
#### QUESTION 2 ####

In [40]:
res = df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime
# print(res.info())
# df_jan[res.isna()].sample() #[['tpep_pickup_datetime', 'tpep_dropoff_datetime']].sample()
(res.dt.total_seconds()/60).std()

np.float64(42.59435124195458)

In [42]:
#### QUESTION 3 ####

In [43]:
df3 = df_jan.copy()
df3['duration_sec'] = (df3.tpep_dropoff_datetime - df3.tpep_pickup_datetime).dt.total_seconds()
df3 = df3[df3['duration_sec'].notna()]

In [49]:
df3 = df3.loc[(df3['duration_sec'] >= 1) & (df3['duration_sec'] <= 60*60)]
print( (len(df3) / len(df_jan)))

0.9917245071844412


In [50]:
#### QUESTION 4 ####